# Turbulent backgrounds with nonlinear OSN model

## Full model
We use a simplified version of the model from Kadakia and Emonet, because OSN adaptation is easy to include into it afterwards. 

The olfactory receptors (OR) have Orco co-receptors. Each OR-Orco complex has an active and an inactive state, with affinities $K^*_{i \mu}$, $K_{i \mu}$ respectively for odor $\mu$, receptor index $i$. Given odor concentrations $c_\mu$, the quasi-static OSN firing rate in response is

$$ A_i = F_\mathrm{max} \left[1 + e^{\epsilon_i(t)} \left( \frac{1 + \sum_\mu K_{i \mu} c_\mu }{1 + \sum_\mu K^*_{i \mu} c_\mu }  \right)  \right]^{-1} $$

where $\epsilon_i(t)$ is the free energy difference between the unbound states in the inactive and active conformations for OR type $i$, in units of $k_\mathrm{B} T$. This free energy difference changes with feedback from OSN activity, with an adaptation time scale of $250$ ms, but here, we will keep $\epsilon_i$ fixed; the point of the model is to have a nonlinear receptor activation with easy inclusion of adaptation later. 

## Simplified form

The full activation function can be simplified in the regime where $K c \ll 1$, which is used by Kadakia and Emonet (in scaled units, they set $K = 0.01$ compared to $K^*$ in the range $10^3-10^4$). Assuming also that $K^* c \gg 1$, we can neglect the $+1$ and simplify to

$$ A_i = F_\mathrm{max}  \frac{ \sum_\mu K^*_{i \mu} c_\mu}{e^{\epsilon_i} + \sum_\mu K^*_{i \mu} c_\mu} $$

which amounts to a Michaelis-Menten function of the linear combination of odor activations, $\sum_\mu K^*_{i \mu} c_\mu$, with Michaelis-Menten constant $e^{\epsilon_i}$ controlling the level of OSN saturation and thus of nonlinearity. Then, odors defined by a single vector of active complex affinities, $\mathbf{K}^*_\mu$. 


## Implementation and affinities distribution

The affinities $K^*_{i \mu}$ and $K_{i \mu}$ are sampled i.i.d. from a distribution of OSN affinities (inverse EC50s) measured experimentally by Si et al., *Neuron*, 2019. We fit this empirical distribution of affinities with a complementary cumulative distribution function of the form

$$ G_X(x) = \mathbb{P}[X > x] = \tanh\left(\frac{1}{b x^{\alpha}} \right) $$

which has a power-law tail $G_X \sim x^{-\alpha}$ for large $x$, but also a cutoff at low $x$ very similar to the experimental data -- much better, at any rate, than a pure power law. See the fits in the notebook ``si2019_hill_tanh_distribution_fits.ipynb``. 

Looking at the code from Kadakia and Emonet, 
https://github.com/elifesciences-publications/ORN-WL-gain-control/blob/master/src/four_state_receptor_CS.py ,
the $K$ parameters are all equal to $1/10^2 = 0.01$ exactly. We do the same, since this ensures each odor has its $K^*$ larger than $K$. Note this puts the model in the large $K^*$, small $K$ regime highlighted above, so $K$ could be neglected entirely if we wanted. 

The free energy default value is $5.0$ in Kadakia and Emonet's code, without variance between odors. 

## Purpose of this notebook
We generate illustrations of the nonlinear manifold and simulation examples of habituation at different levels of nonlinearity. The full simulations to assess performance vs nonlinearity are in the script ``supplementary_scripts/run_performance_nl_osn.py``. We save lines and data for final plotting of the supplementary figure, including:
 - Examples of the simplified OSN activation curves, for different $K^*$ and $\epsilon_i$. 
 - A 2D background manifold plotted in 3D, 
 - The Si et al., 2019 CCDF and the best parameters for the tanh fit are saved in ``si2019_hill_tanh_distribution_fits.ipynb``
 - 

## Functions of general interest

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
from time import perf_counter
import json
from os.path import join as pj

from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    compute_mbars_hgammas_hbargammas,
)
from modelfcts.biopca import (
    integrate_inhib_biopca_network_skip,
    build_lambda_matrix
)
from modelfcts.average_sub import integrate_inhib_average_sub_skip
from modelfcts.average_sub import integrate_inhib_average_sub_skip
from modelfcts.ideal import compute_optimal_matrix_fromsamples
from modelfcts.checktools import analyze_pca_learning
from modelfcts.backgrounds import sample_ss_conc_powerlaw
from modelfcts.distribs import (
    truncexp1_average,
    powerlaw_cutoff_inverse_transform
)
from utils.statistics import seed_from_gen
from simulfcts.plotting import (
    plot_hbars_gamma_series, 
    plot_background_norm_inhibition, 
    plot_pca_results, 
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import l2_norm

In [ ]:
# Main new functions: background update with OSN model
from modelfcts.distribs import inverse_transform_tanhcdf
from modelfcts.nonlin_adapt_osn import (
    generate_odor_tanhcdf, 
    combine_odors_affinities, 
    update_powerlaw_times_concs_affinities
)

## Initialization

### Aesthetic parameters

In [ ]:
do_save_plots = False
do_save_outputs = False

root_dir = ""
outputs_folder = pj(root_dir, "results", "for_plots", "nonlin_adapt")
panels_folder = pj(root_dir, "figures", "nonlin_adapt")
params_folder = pj(root_dir, "results", "common_params")

# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

## Simulation parameters common to all simulations

In [ ]:
# Common parameters for toy and full simulations
inhib_rates = [0.00005, 0.00001]  # alpha, beta  [0.00025, 0.00005]

# Simulation duration
duration = 360000.0
deltat = 1.0

# Common model options
activ_function = "identity"  #"ReLU"

# Background process
combine_fct = combine_odors_affinities
update_fct = update_powerlaw_times_concs_affinities

# Scale of affinity vectors: default
kscale = 5e-4  # default is 5e-4

In [ ]:
def default_background_params(n_comp):
    """ Default time and concentration parameters for the turbulent process"""
    # Turbulent background parameters: same rates and constants for all odors
    back_pms_turbulent = [
        np.asarray([1.0] * n_comp),        # whiff_tmins
        np.asarray([500.] * n_comp),       # whiff_tmaxs
        np.asarray([1.0] * n_comp),        # blank_tmins
        np.asarray([800.0] * n_comp),      # blank_tmaxs
        np.asarray([0.6] * n_comp),        # c0s
        np.asarray([0.5] * n_comp),        # alphas
    ]
    return back_pms_turbulent

In [ ]:
def background_initialization_toy(n_dim, n_comp, rgen, epsil=4.5):
    """ Common template for background initialization """
    # Seed for background simulation, to make sure all models are the same
    simseed = seed_from_gen(rgen)
    back_pms = default_background_params(n_comp)

    # Background odors: epsils controls the nonlinearity strength, will be adjusted
    eps_vec = np.full(n_dim, epsil)
    back_comp = generate_odor_tanhcdf((n_comp, n_dim), rgen, unit_scale=kscale)

    # To keep OSN amplitudes comparable to usual simulations, scale down OSN max. ampli
    avg_whiff = np.mean(truncexp1_average(*back_pms[4:6]))
    
    # Scaling with the max instead of the mean as in full simulations; doesn't matter, 
    # it's just for a cartoonish 2D manifold here. 
    raw_conc = 1.5
    raw_ampli = 5.0
    np_stat = np.amax  # np.mean, np.median, np.amax
    raw_activ = np_stat(combine_fct(np.full(n_comp, raw_conc * avg_whiff), 
                                        back_comp, eps_vec, fmax=1.0))
    osn_ampli = raw_ampli / (raw_activ * np.sqrt(n_dim))

    # Add these extra parameters to the list of background params
    back_pms.append(osn_ampli)
    back_pms.append(eps_vec)
    back_pms.append(back_comp)

    # In the small conc. approx, the odor vectors are (K^* - K)/2
    s_vecs = back_comp / l2_norm(back_comp, axis=1)[:, None]

    # Initial values of background process variables (t, c for each variable)
    init_concs = sample_ss_conc_powerlaw(*back_pms[:-3], size=1, rgen=rgen)
    init_times = powerlaw_cutoff_inverse_transform(
                    rgen.random(size=n_comp), *back_pms[2:4])
    tc_init = np.stack([init_times, init_concs.squeeze()], axis=1)

    # Initial background vector: combine odors with the tc_init concentrations
    init_bkvec = combine_fct(tc_init[:, 1], back_comp, eps_vec, fmax=osn_ampli)
    # nus are first in the list of initial background params
    init_list = [tc_init, init_bkvec]
    
    return back_comp, back_pms, init_list, s_vecs, simseed, osn_ampli

### Optimal linear manifold learning matrix $P$

In [ ]:
def mix_new_back(back_odors, new_odors, cser, newconc, fmax, epsils=5.0):
    n_new = new_odors.shape[0]
    assert n_new == cser.shape[0]  # one new odor per back sample
    all_mixvecs = []
    for n in range(n_new):
        joint_concs = np.concatenate([cser[n], np.full(1, newconc)])
        joint_components = np.concatenate(
            [back_odors, new_odors[n:n+1]], axis=0)
        mixvecs = combine_fct(joint_concs, 
                    joint_components, epsils, fmax=fmax)
        all_mixvecs.append(mixvecs)
    mixvecs = np.stack(all_mixvecs, axis=0)
    return mixvecs

def get_optimal_mat_p(bkvecser, concser, back_pms, new_concs_rel, 
                      sd=0xe20d4b26b1c7e9b943cd23f4c9d15dca):
    """ Compute the optimal linear manifold learning matrix P, 
    using a previously simulated background"""
    avg_whiff_conc = np.mean(truncexp1_average(*back_pms[4:6]))
    new_concs = avg_whiff_conc * new_concs_rel
    n_new_concs = len(new_concs)
    osn_ampli = back_pms[-3]
    eps_vec = back_pms[-2]
    back_comp = back_pms[-1]

    # Compute optimal W matrix for all new odors possible
    # Need samples from the background (use provided bkser)
    # and samples from mixtures of background + new odor
    # (generate from back. conc. series in nuser_ibcm)
    dummy_rgen = np.random.default_rng(sd)
    # New odors, each with a subset of the background samples
    n_samp, n_dims = bkvecser.shape[0], bkvecser.shape[1]
    new_odors_from_distrib = generate_odor_tanhcdf(
        [n_samp, n_dims], dummy_rgen, unit_scale=kscale)

    optimal_matrices = []
    for newconc in new_concs:
        # Mix new odors at newconc with background
        s_new_mix = mix_new_back(back_comp, new_odors_from_distrib, 
                                 concser, newconc, osn_ampli, epsils=eps_vec)
        mat = compute_optimal_matrix_fromsamples(bkvecser, s_new_mix)
        optimal_matrices.append(mat)

    return optimal_matrices

# 2D background manifold illustration
Show also the 2D plane learned by the optimal linear manifold learning matrix $P$. 

We do not need to habituation IBCM, BioPCA, etc. on this manifold: we can just do an average subtraction simulation, extract the background series, compute the optimal $P$ for that background and random new odors. 

In [ ]:
# Background initialization is specific to this simulation
n_dimensions_toy = 25  
n_components_toy = 2  # Number of background odors

skp = 20 * int(1.0 / deltat)

# Choose randomly generated background vectors
rgen_meta_toy = np.random.default_rng(seed=0x220369e90599ffa80a743d99ac942f28)

res = background_initialization_toy(n_dimensions_toy, n_components_toy, rgen_meta_toy, epsil=4.5)
(back_components_toy, back_params_toy, init_bk_list_toy, 
         s_gamma_vecs_toy, simul_seed_toy, osn_ampli_toy) = res

In [ ]:
# Dummy average subtraction simulation, to get a background sample
avg_options = {"activ_fct": activ_function}

# Initial synaptic weights: dummy
init_avg_toy = np.zeros([1, n_dimensions_toy])

sim_results = integrate_inhib_average_sub_skip(
                init_avg_toy, update_fct, init_bk_list_toy, 
                [], inhib_rates, back_params_toy, duration, deltat,
                seed=simul_seed_toy, noisetype="uniform", skp=skp*2, **avg_options
)
tser_toy, tcser_toy, bkvecser_toy, _, _ = sim_results

In [ ]:
# Compute optimal matrix P for this process
optim_p = get_optimal_mat_p(bkvecser_toy, tcser_toy[:, :, 1], back_params_toy, np.ones(1), 
                      sd=0xe20d4b26b1c7e9b943cd23f4c9d15dca)[0]

In [ ]:
# Plot 2D manifold in a 3D slice,
dims = (0, 2, 4)
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
where_01 = (tcser_toy[:, :, 1] > 0).astype(bool)
locations = {
    "Both odors": np.all(where_01, axis=1),  #all
    "Odor 0": (where_01[:, 0] & ~where_01[:, 1]),  # 0
    "Odor 1": (~where_01[:, 0] & where_01[:, 1])  # 1
}
all_colors = {
    "Both odors": "xkcd:purple",
    "Odor 0": "xkcd:blue",
    "Odor 1": "xkcd:red",

}

# Background odors plane
vecs = np.zeros(s_gamma_vecs_toy.shape)
scale = 2.0
orig = np.zeros([3, n_components_toy])
for i in range(n_components_toy):
    vecs[i] = s_gamma_vecs_toy[i] / np.sqrt(np.sum(s_gamma_vecs_toy[i]**2)) * scale
    
scale2 = 1.75
topvecs = vecs.T
s1, s2 = np.meshgrid(np.arange(0, 1, 0.2)*scale2, np.arange(0, 1, 0.2)*scale2)
plane = s1[None, :, :] * topvecs[dims, 0:1, None] + s2[None, :, :] * topvecs[dims, 1:2, None]
x, y, z = plane[0], plane[1], plane[2]
ls = mpl.colors.LightSource(170, 45)
rgb = ls.shade(z, cmap=mpl.cm.Greys, vert_exag=0.1, blend_mode='soft')
#ax.plot_surface(x, y, z-0.05, color="grey", alpha=0.3, linewidth=0.5, lightsource=ls)
ax.plot_surface(x, y, z-0.05, color="grey", alpha=0.3, rstride=1, cstride=1, facecolors=rgb,
                       linewidth=0, antialiased=False, shade=True)

for lbl in ["Odor 0", "Odor 1", "Both odors"]:
    slc = locations[lbl]
    ax.scatter(bkvecser_toy[slc, dims[0]], bkvecser_toy[slc, dims[1]], 
               bkvecser_toy[slc, dims[2]], s=4, lw=0.3, label=lbl, color=all_colors[lbl])

ax.quiver(*orig, *(vecs[:, dims].T), color="k", lw=1.5, arrow_length_ratio=0.2)
ax.scatter(0, 0, 0, color="k", s=25)

# Labeling
for lbl, f in enumerate([ax.set_xlabel, ax.set_ylabel, ax.set_zlabel]):
    zlbl = f("OSN {}".format(lbl+1), labelpad=-17.5)
for f in [ax.set_xticks, ax.set_yticks, ax.set_zticks]:
    f([])
for f in [ax.set_xticklabels, ax.set_yticklabels, ax.set_zticklabels]:
    f([], pad=0.1)

ax.view_init(azim=240, elev=35)

#leg = ax.legend(loc="upper right", bbox_to_anchor=(0.0, 1.0), frameon=False)
fig.tight_layout()

# Need to adjust the tightbox to remove whitespace above and below manually. 
#ax.set_aspect("equal")
fig.tight_layout()
tightbox = fig.get_tightbbox()
tightbox._bbox.y0 = tightbox._bbox.y0*1.6
tightbox._bbox.y1 = tightbox._bbox.y1 - 0.8*tightbox._bbox.y0
tightbox._bbox.x0 = tightbox._bbox.x0 * 0.7

figname = "background_manifold_2d_example.pdf"
if do_save_plots:
    fig.savefig(pj("figures", "nonlin_adapt", figname), 
                transparent=True, bbox_inches=tightbox, bbox_extra_artists=(zlbl, leg))
plt.show()
plt.close()

In [ ]:
# Save samples and surface
if do_save_outputs:
    fname = pj(outputs_folder, "2d_manifold_nonlinear_osn.npz")
    np.savez_compressed(fname, conc_ser=tcser_toy[:, :, 1], bkvecser=bkvecser_toy[:, dims], 
                        plane=plane, vecs=s_gamma_vecs_toy[:, dims]*scale)

# Full simulation examples at different $\epsilon$s

## Global model parameters for full simulations

We will run simulations for different $\epsilon$ values, with adjusted OSN amplitudes, so place most of the background odors generation in a simulation initialization function. 

In [ ]:
# Initialize common simulation parameters
n_dimensions = 50  # Fly number
n_components = 6  # Number of background odors

# Simulation skipping, 50 is enough for plots
skp = 50 * int(1.0 / deltat)

## Background initialization

In [ ]:
def initialize_back_params(epsil, rgen, n_comp, n_dim):
    # Turbulent background parameters: same rates and constants for all odors
    back_pms = default_background_params(n_comp)

    epsils_vec = np.full(n_dimensions, epsil)
    back_comps = generate_odor_tanhcdf((n_comp, n_dim), rgen, unit_scale=kscale)

    # To keep OSN amplitudes comparable to usual simulations, scale down OSN max. ampli
    avg_whiff_conc = np.mean(truncexp1_average(*back_pms[4:6]))
    
    # Same adjustment of the OSN amplitude as in the performance recognition tests
    raw_conc_factor = 2.5
    raw_ampli = 2.5
    np_statistic = np.mean  # np.mean, np.median, np.amax

    raw_osn_activ = np_statistic(combine_fct(np.full(n_comp, raw_conc_factor * avg_whiff_conc), 
                                        back_comps, epsils_vec, fmax=1.0))
    max_osn_ampli = raw_ampli / (raw_osn_activ * np.sqrt(n_dim))

    # Add these extra parameters to the list of background params
    back_pms.append(max_osn_ampli)
    back_pms.append(epsils_vec)
    back_pms.append(back_comps)

    # Initialization
    # Initial values of background process variables (t, c for each variable)
    init_concs = sample_ss_conc_powerlaw(*back_pms[:-3], size=1, rgen=rgen)
    init_times = powerlaw_cutoff_inverse_transform(
                    rgen.random(size=n_comp), *back_pms[2:4])
    tc_init = np.stack([init_times, init_concs.squeeze()], axis=1)

    # Initial background vector: combine odors with the tc_init concentrations
    init_bkvec = combine_fct(tc_init[:, 1], back_comps, epsils_vec, fmax=max_osn_ampli)
    # nus are first in the list of initial background params
    init_back = [tc_init, init_bkvec]
    
    return back_pms, init_back

## IBCM habituation simulations and parameters

In [ ]:
# IBCM model parameters, same for each tested epsilon
n_i_ibcm = 24  # Number of inhibitory neurons for IBCM case

# Model rates
learnrate_ibcm = 0.00075  #5e-5
tau_avg_ibcm = 1600  # 2000
coupling_eta_ibcm = 0.7/n_i_ibcm
ssat_ibcm = 50.0
k_c2bar_avg = 0.1
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "tanh", 
    "variant": "law", 
    "decay": True
}

### Function to run and clean a simulation at a given $\epsilon$

Uses global IBCM parameters defined above. 

In [ ]:
def run_ibcm_simulation_epsil(epsil, n_comp, n_dim, rgenseed, simseed, skp_local=skp):
    print("Initializing IBCM simulation for epsilon =", epsil)
    # Initialize background with the random generator with seed rgenseed
    rgen = np.random.default_rng(rgenseed)
    res = initialize_back_params(epsil, rgen, n_comp, n_dim)
    back_params_local, init_back = res
    
    # Initial synaptic weights: small positive noise
    init_synapses_ibcm = 0.2*rgen.standard_normal(size=[n_i_ibcm, n_dim])*lambd_ibcm
    
    # Run the IBCM simulation
    print("Starting IBCM simulation...")
    tstart = perf_counter()
    sim_results = integrate_inhib_ibcm_network_options(
                init_synapses_ibcm, update_fct, init_back, 
                ibcm_rates, inhib_rates, back_params_local, duration, 
                deltat, seed=simseed, noisetype="uniform",  
                skp=skp_local, **ibcm_options
    )
    tend = perf_counter()
    print("Finished IBCM simulation for epsilon =", epsil, "in {:.2f} s".format(tend - tstart))
    
    return back_params_local, sim_results

### Cleaning and saving functions for IBCM

In [ ]:
def mix_new_odors_in_manifold(back_pms, conc_ser, new_conc_rel, rgen, n_ex=2, n_samp=10):
    """Mix n_ex new odors with n_samp background samples each. 
    Returns a 3d-array of mixtures, indexed [n_ex, n_samp, n_dim], 
    and the new odor vectors, a 2d array indexed [n_ex, n_dim]. 
    """
    back_odors = back_pms[-1]
    n_comp, n_dim = back_odors.shape[0], back_odors.shape[1]
    max_ampli = back_pms[-3]
    print(max_ampli)
    new_odors = generate_odor_tanhcdf((n_ex, n_dim), rgen, unit_scale=kscale)
    avg_whiff_conc = np.mean(truncexp1_average(*back_pms[4:6]))
    new_conc = avg_whiff_conc * new_conc_rel
    non_null_concs = conc_ser[np.any(conc_ser > 0.0, axis=1)]
    epsils_vec = np.full(n_dim, back_pms[-2])
    back_concs = non_null_concs[rgen.choice(non_null_concs.shape[0], size=n_ex*n_samp, replace=True)]
    back_concs = back_concs.reshape(n_ex, n_samp, n_comp)
    all_mixed_samples = []
    for i in range(n_ex):
        joint_kmats = np.concatenate([back_odors, new_odors[i:i+1]], axis=0)
        mixed_samples_i = []
        for j in range(n_samp):
            joint_concs = np.concatenate([back_concs[i, j:j+1], np.full((1, 1), new_conc)], axis=1)
            mixed_samples_i.append(combine_fct(joint_concs, joint_kmats, epsils_vec, fmax=max_ampli))
        mixed_samples_i = np.concatenate(mixed_samples_i, axis=0)
        all_mixed_samples.append(mixed_samples_i)
        
    return np.stack(all_mixed_samples, axis=0), new_odors

In [ ]:
def analyze_clean_ibcm_simul(results_raw, back_pms, rgenseed, n_ex=2, n_samp=10):
    """
    Args:
        results_raw = (tser_ibcm, nuser_ibcm, bkvecser_ibcm, mser_ibcm, 
            cbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm)
    Returns:
        cbars_gamma, wser_ibcm, bkvecser_ibcm, 
            yser_ibcm, moments_conc, cgammas_bar_counts, specif_gammas, correl_c_conc
    """
    (tser_ibcm, nuser_ibcm, bkvecser_ibcm, mser_ibcm, 
        cbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm) = results_raw
    # Calculate cgammas_bar and mbars
    transient = int(5/6*duration / deltat) // skp
    back_components = back_pms[-1]
    epsil = back_pms[-2]
    basis = back_components / l2_norm(back_components, axis=1)[:, None] 

    # Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
    mbarser, c_gammas, cbars_gamma = compute_mbars_hgammas_hbargammas(
                                results_raw[3], coupling_eta_ibcm, basis)
    
    # Moments of concentrations
    conc_ser = nuser_ibcm[:, :, 1]
    mean_conc = np.mean(conc_ser)
    sigma2_conc = np.var(conc_ser)
    thirdmom_conc = np.mean((conc_ser - mean_conc)**3)
    moments_conc = [float(mean_conc), float(sigma2_conc), float(thirdmom_conc)]

    # Count how many dot products are at each possible value. Use cbar = 1.0 as a split. 
    cbars_gamma_mean = np.mean(cbars_gamma[transient:], axis=0)
    specif_gammas = np.argmax(np.mean(cbars_gamma[transient:], axis=0), axis=1)
    
    cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
    conc_ser_centered = conc_ser - np.mean(conc_ser[transient:], axis=0)
    correl_c_conc = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)
    
    ysernorm_ibcm = l2_norm(yser_ibcm, axis=1)
    
    # Examples of mixing new odors with the background
    rgen = np.random.default_rng(np.random.SeedSequence(rgenseed).spawn(2)[1])
    mixres = mix_new_odors_in_manifold(back_pms, conc_ser, 1.0, rgen, n_ex=n_ex, n_samp=n_samp)
    mixed_samples, new_odors = mixres
    results_clean = (cbars_gamma, wser_ibcm, bkvecser_ibcm, ysernorm_ibcm, moments_conc, 
                     cbars_gamma_mean, specif_gammas, correl_c_conc, back_components, 
                     conc_ser, mixed_samples, new_odors)
    return results_clean


def save_ibcm_simuls_to_disk(fname, **all_results_clean):
    # Save cbar gamma series, background series, ynorm series
    # Will run a separate short, non-skipped simulation to plot mixed concentrations
    all_saved_series = {}
    for simname in all_results_clean.keys():
        (cbars_gamma, _, bkvecser_ibcm, ysernorm_ibcm, _, _, _, _, 
             back_comps, conc_ser, mixed_samples, new_odors) = all_results_clean[simname]
        fullname = "cbars_gamma_ser_" + simname
        all_saved_series[fullname] = cbars_gamma
        # For habituation and manifold plots, save back series
        # and odor components
        fullname = "bkvec_ser_" + simname
        all_saved_series[fullname] = bkvecser_ibcm
        fullname = "back_components_" + simname
        all_saved_series[fullname] = back_comps
        fullname = "conc_ser_" + simname
        all_saved_series[fullname] = conc_ser
        # For habituation plots, save norm of y
        fullname = "y_norm_ser_" + simname
        all_saved_series[fullname] = ysernorm_ibcm
        # Mixture of new odors with background odors, within the manifold
        fullname = "mixed_new_odors_" + simname
        all_saved_series[fullname] = mixed_samples
        fullname = "new_odors_" + simname
        all_saved_series[fullname] = new_odors
    np.savez_compressed(fname, **all_saved_series)
    return 0

In [ ]:
# Plotting functions for IBCM
def plot_ibcm_results(res_ibcm_raw, res_ibcm_clean):
    (cbars_gamma, wser_ibcm, bkvecser_ibcm, ysernorm_ibcm, 
         moments_conc, cbars_gamma_mean, specif_gammas, correl_c_conc, 
         back_comps, conc_ser, _, _) = res_ibcm_clean

    # Plot of cbars gamma series
    fig , ax, _ = plot_hbars_gamma_series(tser_common, cbars_gamma, 
                            skp=2, transient=320000 // skp)
    fig.tight_layout()
    leg = ax.legend(loc="upper left", bbox_to_anchor=(1., 1.))
    plt.show()
    plt.close()


    # Plots of neuron specificities
    fig, ax = plt.subplots()
    img = ax.imshow(correl_c_conc.T)
    ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
    fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
                 r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
                location="top")
    fig.tight_layout()
    plt.show()
    plt.close()

    # Check if each component has at least one neuron
    print("Odor specificities:", specif_gammas)
    split_val = 2.5
    for comp in range(n_components):
        print("Number of neurons specific to component {}: {}".format(
                comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

    # Plot of background inhibition
    fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                    tser_common, res_ibcm_raw[2], res_ibcm_raw[7], skp=2)

    # Compute noise reduction factor, annotate
    transient = 250000 // skp
    norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

    print("Mean activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
    print("Standard deviation of activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
    ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
               xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

    ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
    fig.tight_layout()
    plt.show()
    plt.close()

## BioPCA initialization and simulation functions

In [ ]:
# BioPCA model parameters, same for all epsilons
n_i_pca = n_components * 2  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 1e-4  # Learning rate of M
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
# but scale it up to counteract W regularization
lambda_range_pca = 0.5
lambda_max_pca = 9.0
# Learning rate of L, relative to learnrate. Adjusted to Lambda in the integration function
rel_lrate_pca = 2.0  #  / lambda_max_pca**2 
lambda_mat_diag = build_lambda_matrix(lambda_max_pca, lambda_range_pca, n_i_pca)

xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_max_pca, lambda_range_pca, xavg_rate_pca]

In [ ]:
def run_biopca_simulation_epsil(epsil, n_comp, n_dim, rgenseed, simseed, skp_local=skp):
    print("Initializing BioPCA simulation for epsilon =", epsil)
    # Initialize background parameters, give same rgenseed as IBCM to have same background
    rgen = np.random.default_rng(rgenseed)
    res = initialize_back_params(epsil, rgen, n_comp, n_dim)
    back_params_local, init_back = res
        
    init_synapses_pca = rgen.standard_normal(size=[n_i_pca, n_dim]) / np.sqrt(n_i_pca)
    init_mmat_pca = rgen.standard_normal(size=[n_i_pca, n_dim]) / np.sqrt(n_dim)
    init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
    ml_inits_pca = [init_mmat_pca, init_lmat_pca]
    
    # Run the IBCM simulation
    print("Starting BioPCA simulation...")
    tstart = perf_counter()
    sim_results = integrate_inhib_biopca_network_skip(
                ml_inits_pca, update_fct, init_back, biopca_rates, 
                inhib_rates, back_params_local, duration, deltat, 
                seed=simseed, noisetype="uniform", skp=skp_local, **pca_options
    )
    tend = perf_counter()
    print("Finished BioPCA simulation for epsilon =", epsil, "in {:.2f} s".format(tend - tstart))
    
    return back_params_local, sim_results

### Cleaning and saving functions for BioPCA

In [ ]:
def analyze_clean_biopca_simul(results_raw):
    """
    We do not need to save odor vectors (back_components), 
    since the IBCM simulation will provide them for both models. 
    
    Args:
        results_raw = (tser_pca, nuser_pca, bkvecser_pca, mser_pca, 
            lser_pca, xser_pca, cbarser_pca, wser_pca, yser_pca)
    Returns:
        bkvecser_pca, ysernorm_pca, wser_pca, true_pca, 
            learnt_pca, off_diag_l_avg_abs, align_error_ser)
    """
    (tser_pca, nuser_pca, bkvecser_pca, mser_pca, lser_pca, xser_pca, 
         cbarser_pca, wser_pca, yser_pca) = results_raw
    
    # Analyze versus true offline PCA of the background samples
    print("Starting analysis of BioPCA vs true PCA")
    tstart = perf_counter()
    res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
    true_pca, learnt_pca, _, off_diag_l_avg_abs, align_error_ser = res
    tend = perf_counter()
    print("Completed analysis in {:.1f} s".format(tend - tstart))
    
    ysernorm_pca = l2_norm(yser_pca, axis=1)
    bkvecsernorm_pca = l2_norm(bkvecser_pca, axis=1)
    
    # Also save info about background vs yser_pca
    results_clean = (bkvecsernorm_pca, ysernorm_pca, wser_pca,
                     true_pca, learnt_pca, off_diag_l_avg_abs, align_error_ser)
    return results_clean


def save_biopca_simuls_to_disk(fname, **all_results_clean):
    # Save true and learnt PCA, that's all we really need
    true_learnt_pcas = {}
    for simname in all_results_clean.keys():
        (bkvecsernorm_pca, ysernorm_pca, wser_pca, true_pca, 
         learnt_pca, off_diag_l_avg_abs, align_error_ser) = all_results_clean[simname]
        fullname = "true_pca_vals_" + simname
        true_learnt_pcas[fullname] = true_pca[0]
        fullname = "learnt_pca_vals_" + simname
        true_learnt_pcas[fullname] = learnt_pca[0]
        fullname = "pca_align_error_" + simname
        true_learnt_pcas[fullname] = align_error_ser
        fullname = "bkvec_norm_ser_" + simname
        true_learnt_pcas[fullname] = bkvecsernorm_pca
        fullname = "y_norm_ser_" + simname
        true_learnt_pcas[fullname] = ysernorm_pca
        print(learnt_pca[1].shape)
    np.savez_compressed(fname, **true_learnt_pcas)
    return 0


In [ ]:
def plot_biopca_results(res_biopca_raw, res_biopca_clean):
    (bkvecsernorm_pca, ysernorm_pca, wser_pca, true_pca, 
    learnt_pca, off_diag_l_avg_abs, align_error_ser) = res_biopca_clean

    # Plot learnt vs true PCA
    fig, axes = plot_pca_results(tser_common/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
    axes[-1].set_xlabel("Time (x1000 steps)")
    axes[0].get_legend().remove()
    fig.tight_layout()
    fig.set_size_inches(fig.get_size_inches()[0], 2.5*plt.rcParams["figure.figsize"][1])
    plt.show()
    plt.close()

    # Plot level of background inhibition
    fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                    tser_common, res_biopca_raw[2], res_biopca_raw[8], skp=2)

    # Compute noise reduction factor, annotate
    transient = 250000 // skp
    norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

    print("Mean activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
    print("Standard deviation of activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
    ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
               xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

    ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
    fig.tight_layout()
    plt.show()
    plt.close()

## IBCM simulations in the near-linear regime ($\epsilon = 10$)

BioPCA usually converges to a decent basis in these conditions, but IBCM is more sensitive to strong outlier odors and needs rates to be well adjusted care. 

In [ ]:
tser_common = np.arange(0.0, duration, deltat*skp)

In [ ]:
epsil_low = 10.0
# Good seeds for mean, ampli=2.5, conc_factor = 2.5  -- this seems to be a winning combination
#seed_low = 0xe3a6a312d14ebc2e72e08619bc830375
#simul_seed_low = 0x2ac13b6a33b7a947037ab8c07b9ecb8
#seed_low = 0xe3a6a312d14ebc2e72e08619bc8f017e
#simul_seed_low = 0x2ac13b6a30b7a947037ab8c37b9edb0
#seed_low = 0x3180b82d0efbefd8131def347e3a99e6
#simul_seed_low = 0xb4ebbf94d1f271498ccde6a094db41fe
#seed_low = 0x3170b82d0efbefd8131def347e3a93e5
#simul_seed_low = 0xb4e7bf94e1f271398ccde6a094db44fc
#seed_low = 0x3270b82d0efb6fd8131def555e3a7341
#simul_seed_low = 0x53e7bfc4e1f84139833de6a094db14fc
#seed_low = 0x7170b82d90fb6fd8331def666e3a7343
#simul_seed_low = 0x52e7bfc4e1f58395730de6a091db14fe
#seed_low = 0x7170b82d905839fffa1def666e3a7343
#simul_seed_low = 0x52e7bfc4e1f58395730de6afff855abc

# Bad seeds showing sensitivity to odor asymmetry, one odor not covered
#seed_low = 0x3170b82d0efbefd8131def347e3a93e3
#simul_seed_low = 0xb4e7bf94e1f271398ccde6a094db14fc
#seed_low = 0x7170b82d0efb6fd8131def555e3a7344
#simul_seed_low = 0x52e7bfc4e1f84139833de6a094db14fe

seed_low = 0x7170b82d905839fffa1def666e3a7343
simul_seed_low = 0x52e7bfc4e1f58395730de6afff855abc

# IBCM
back_ibcm_low, res_ibcm_low = run_ibcm_simulation_epsil(epsil_low, n_components,  
                         n_dimensions, seed_low, simul_seed_low)
res_ibcm_clean_low = analyze_clean_ibcm_simul(res_ibcm_low, back_ibcm_low, seed_low)

# BioPCA
back_biopca_low, res_biopca_low = run_biopca_simulation_epsil(epsil_low, n_components,
                            n_dimensions, seed_low, simul_seed_low)
res_biopca_clean_low = analyze_clean_biopca_simul(res_biopca_low)

In [ ]:
plot_ibcm_results(res_ibcm_low, res_ibcm_clean_low)

In [ ]:
plot_biopca_results(res_biopca_low, res_biopca_clean_low)

## Run simulations at weak nonlinearity (moderate $\epsilon = 6.0$)

In [ ]:
epsil_moderate = 6.0
# Good seeds for mean, ampli=2.5, conc_factor = 2.5
#seed_moderate = 0x3e94bc0db982c06f17bf8db6e2a0afd5
#simul_seed_moderate = 0x95aeaedb6bca5aabb0e69e132ac9d553
#seed_moderate = 0xb05fac69bcde76cabe7e42d06f7a9cad
#simul_seed_moderate = 0xe3484614b86cfb179feef21cb9b5e5e2
#seed_moderate = 0x5426f8cb35b1ad2c4bfbc0d1f9abdcee
#simul_seed_moderate = 0x6cc1cb582e512808964f71f0edf226d
#seed_moderate = 0x9996f8cb35b1ad2c3ee4c1d1f994dccc
#simul_seed_moderate = 0xfff1cb582e512808964f71f0edf335d
# The pair below is good for epsilon = 6.5 and epsilon = 6.0 too
#seed_moderate = 0x4b10b81800bf92f1ad2bbca95f6beca8
#simul_seed_moderate = 0x9c535fa8057f20f5381d1fb9f485c680

# Bad seeds for mean, ampli=2.5, conc_factor = 2.5
#seed_moderate = 0xb27a42505b3ee99ea8162c0d4127b11
#simul_seed_moderate = 0x68e595fb1df3fb9c75d975a6682b9c78
#seed_moderate = 0xb27a42505b3ee99ea8162c0d5839ffb
#simul_seed_moderate = 0x68e595fb1df3fb9c75d975a6682b9cad
#seed_moderate = 0x5ab03d3362deb1715e30951ebc6fbf86
#simul_seed_moderate = 0x94b586389cc1abcafd984e13b85b001b
#seed_moderate = 0x5ab03d3362deb1715e30951ebc90ebec
#simul_seed_moderate = 0x94b586389cc1bccadd984e13b85b754e
#seed_moderate = 0x5426f8cb35b1ad2c4ee4c0d1f994dcee
#simul_seed_moderate = 0x6cc1cb582e512808964f71f0edf335f

seed_moderate = 0x9996f8cb35b1ad2c3ee4c1d1f994dccc
simul_seed_moderate = 0xfff1cb582e512808964f71f0edf335d



# IBCM
back_ibcm_mod, res_ibcm_mod = run_ibcm_simulation_epsil(epsil_moderate, n_components,  
                         n_dimensions, seed_moderate, simul_seed_moderate)
res_ibcm_clean_moderate = analyze_clean_ibcm_simul(res_ibcm_mod, back_ibcm_mod, seed_moderate)

# BioPCA
back_biopca_mod, res_biopca_mod = run_biopca_simulation_epsil(epsil_moderate, n_components,
                            n_dimensions, seed_moderate, simul_seed_moderate)
res_biopca_clean_moderate = analyze_clean_biopca_simul(res_biopca_mod)

In [ ]:
plot_ibcm_results(res_ibcm_mod, res_ibcm_clean_moderate)

In [ ]:
plot_biopca_results(res_biopca_mod, res_biopca_clean_moderate)

## Run simulations at strong nonlinearity (small $\epsilon = 3.5$)

In [ ]:
epsil_strong = 3.5
seed_strong = 0xb57184f5b6a9d2d998b4fb116f366ef1
simul_seed_strong = 0x424f10cb5dce0d8c10b31bd7d818e130


# IBCM
back_ibcm_strong, res_ibcm_strong = run_ibcm_simulation_epsil(epsil_strong, n_components,  
                         n_dimensions, seed_strong, simul_seed_strong)
res_ibcm_clean_strong = analyze_clean_ibcm_simul(res_ibcm_strong, back_ibcm_strong, seed_strong)

# BioPCA
back_biopca_strong, res_biopca_strong = run_biopca_simulation_epsil(epsil_strong, n_components,
                            n_dimensions, seed_strong, simul_seed_strong)
res_biopca_clean_strong = analyze_clean_biopca_simul(res_biopca_strong)

In [ ]:
plot_ibcm_results(res_ibcm_strong, res_ibcm_clean_strong)

In [ ]:
plot_biopca_results(res_biopca_strong, res_biopca_clean_strong)

## Case at large $\epsilon$ (low nonlinearity) where IBCM converges too fast

In [ ]:
epsil_low_fail = 10.0
#seed_low_fail = 0x7170b82d0efb6fd8131def555e3a7344
#simul_seed_low_fail = 0x52e7bfc4e1f84139833de6a094db14fe
seed_low_fail = 0x3170b82d0efbefd8131def347e3a93e3
simul_seed_low_fail = 0xb4e7bf94e1f271398ccde6a094db14fc

# IBCM
back_ibcm_low_fail, res_ibcm_low_fail = run_ibcm_simulation_epsil(epsil_low_fail, n_components,  
                         n_dimensions, seed_low_fail, simul_seed_low_fail)
res_ibcm_clean_low_fail = analyze_clean_ibcm_simul(res_ibcm_low_fail, back_ibcm_low_fail, seed_low_fail)

# BioPCA
back_biopca_low_fail, res_biopca_low_fail = run_biopca_simulation_epsil(epsil_low_fail, n_components,
                            n_dimensions, seed_low_fail, simul_seed_low_fail)
res_biopca_clean_low_fail = analyze_clean_biopca_simul(res_biopca_low_fail)

In [ ]:
plot_ibcm_results(res_ibcm_low_fail, res_ibcm_clean_low_fail)

In [ ]:
plot_biopca_results(res_biopca_low_fail, res_biopca_clean_low_fail)

# Habituation by optimal linear manifold learning matrix $P$

In [ ]:
def mix_new_back(back_pms, bkser, cser, new_odors, newconc):
    back_odors = back_pms[-1]
    epsils = back_pms[-2]
    max_ampli = back_pms[-3]
    n_new = new_odors.shape[0]
    assert n_new == cser.shape[0]  # one new odor per back sample
    all_mixvecs = []
    for n in range(n_new):
        joint_concs = np.concatenate([cser[n], np.full(1, newconc)])
        joint_components = np.concatenate([back_odors, new_odors[n:n+1]], axis=0)
        mixvecs = combine_fct(joint_concs, joint_components, epsils, fmax=max_ampli)
        all_mixvecs.append(mixvecs)
    mixvecs = np.stack(all_mixvecs, axis=0)
    return mixvecs

In [ ]:
def optimal_habituation(back_pms, bkser, cser, rgen, newconc_rel):
    # Compute optimal W matrix for all new odors possible
    # Need samples from the background (use bkvecser_ibcm)
    # and samples from mixtures of background + new odor
    # (generate from back. conc. series in nuser_ibcm)
    # New odors, each with a subset of the background samples
    avg_whiff_conc = np.mean(truncexp1_average(*back_pms[4:6]))
    newconc = newconc_rel * avg_whiff_conc
    n_dim = bkser.shape[1]
    newods = generate_odor_tanhcdf([bkser.shape[0], bkser.shape[1]], rgen, unit_scale=kscale)
    s_new_mix = mix_new_back(back_pms, bkser, cser, newods, newconc)
    optimal_matrix = compute_optimal_matrix_fromsamples(bkser, s_new_mix)

    # Use the W matrix for the lowest concentration to inhibit the background
    yser_optimal = bkser - bkser.dot(optimal_matrix.T)
    return yser_optimal

In [ ]:
# Compute optimal y series for each simulation above
rgen_optim = np.random.default_rng(0xc6ede9d7cf780703439a669234fcb8fd)
y_optim_low = optimal_habituation(back_ibcm_low, res_ibcm_clean_low[2],
                         res_ibcm_clean_low[9], rgen_optim, 1.0)
y_optim_low_fail = optimal_habituation(back_ibcm_low_fail, res_ibcm_clean_low_fail[2],
                         res_ibcm_clean_low_fail[9], rgen_optim, 1.0)
y_optim_moderate = optimal_habituation(back_ibcm_mod, res_ibcm_clean_moderate[2], 
                        res_ibcm_clean_moderate[9], rgen_optim, 1.0)
y_optim_strong = optimal_habituation(back_ibcm_strong, res_ibcm_clean_strong[2], 
                            res_ibcm_clean_strong[9], rgen_optim, 1.0)
all_results_y_optimal = {
    "ynorm_ser_" + str(epsil_low): l2_norm(y_optim_low, axis=1),
    "ynorm_ser_" + str(epsil_low_fail): l2_norm(y_optim_low_fail, axis=1),
    "ynorm_ser_" + str(epsil_moderate): l2_norm(y_optim_moderate, axis=1),
    "ynorm_ser_" + str(epsil_strong): l2_norm(y_optim_strong, axis=1)
}

# Save results for final plotting

In [ ]:
fname_ibcm = pj(outputs_folder, "saved_ibcm_simulations_nonlin_osn.npz")
fname_biopca = pj(outputs_folder, "saved_biopca_simulations_nonlin_osn.npz")
fname_optimal = pj(outputs_folder, "saved_optimal_habituation_nonlin_osn.npz")
# Saving examples where the models work
all_results_clean_ibcm = {
    str(epsil_strong): res_ibcm_clean_strong, 
    str(epsil_moderate): res_ibcm_clean_moderate,
    str(epsil_low_fail): res_ibcm_clean_low_fail
}
all_results_clean_biopca = {
    str(epsil_strong): res_biopca_clean_strong, 
    str(epsil_moderate): res_biopca_clean_moderate,
    str(epsil_low_fail): res_biopca_clean_low_fail
}

if do_save_outputs:
    save_ibcm_simuls_to_disk(fname_ibcm, **all_results_clean_ibcm)
    save_biopca_simuls_to_disk(fname_biopca, **all_results_clean_biopca)
    np.savez_compressed(fname_optimal, **all_results_y_optimal)